In [ ]:
from pathlib import Path
from collections import Counter
import re
import pickle

corpus_dir = Path(dir_manager.get_data_dir("corpus"))

In [ ]:
word_use = Counter()
word_doc = Counter()

newline_dash = re.compile(r"(\S+)-\s+(\S+)")
for ix, fpath in enumerate(corpus_dir.glob("*/EN_TXT_ORIG/*.txt")):

    with open(fpath, "rb") as open_file:
        text = open_file.read().decode("utf-8", errors="ignore")
        text = newline_dash.sub(r"\1\2", text).lower()
        doc_use = Counter(re.findall(r"[a-z]+", text))
        for word in doc_use:
            word_doc[word] += 1

    word_use.update(doc_use)

with open(corpus_dir / "en_corpus_word_use.dict.pickle", "wb") as open_file:
    pickle.dump(word_use, open_file)

with open(corpus_dir / "en_corpus_word_doc.dict.pickle", "wb") as open_file:
    pickle.dump(word_doc, open_file)

In [ ]:
import enchant
import re
VALID_TOKEN_PAT = re.compile("^[a-z]+$")
en_dict = enchant.Dict("en_US")

def check(word):
    is_en = en_dict.check(word)
    if not is_en:
        s = en_dict.suggest(word)
        is_en = word.lower() in {i.lower() for i in s}
    return is_en

In [ ]:
word_doc_df = pd.Series(word_doc).sort_values(ascending=False)
word_doc_df.name = "freq"

word_use_df = pd.Series(word_use).sort_values(ascending=False)
word_use_df.name = "freq"

word_usage_rate = word_use_df / word_doc_df

# Prerequisite for next step: Wikipedia word list

1. `cd /data/wb536061 && git clone https://github.com/IlyaSemenov/wikipedia-word-frequency.git && cd wikipedia-word-frequency`
2. Download latest wiki data from [https://dumps.wikimedia.org/enwiki/latest/](https://dumps.wikimedia.org/enwiki/latest/) :: [https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2](https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2) to /data/wb536061/wb_nlp/data/external/wikipedia/
    1. Download in VM then use WinSCP to transfer the data to the server
3. Run `./gather_wordfreq.py ../wb_nlp/data/external/wikipedia/enwiki-latest-pages-articles.xml.bz2 > wordfreq-enwiki-latest-pages-articles.xml.bz2.txt`
4. Copy data to repo `cp wordfreq-enwiki-latest-pages-articles.xml.bz2.txt ~/wb_nlp/data/whitelists/whitelists/`
5. Add and commit to repo.

In [ ]:
wiki_count = pd.read_csv(
    dir_manager.get_data_dir("whitelists", "whitelists", "wordfreq-enwiki-latest-pages-articles.xml.bz2.txt"),
    sep=" ", names=["word", "freq"])

In [ ]:
word_doc_frame = word_doc_df.to_frame().head(100000)
word_doc_frame["en"] = word_doc_frame.index.map(check)
word_doc_frame.head()

In [ ]:
w = word_doc_frame[word_doc_frame["en"] == False].index.intersection(wiki_count.head(500000)["word"])

In [ ]:
with open(dir_manager.get_data_dir("whitelists", "whitelists", "doc-freq-wiki-wordlist.txt"), "w") as open_file:
    for i in w:
        if VALID_TOKEN_PAT.match(i):
            open_file.write(i.strip() + "\n")

In [ ]:
oov_words = word_doc_df.index.difference(wiki_count.word)